In [2]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from google.transit import gtfs_realtime_pb2
from pprint import pprint

%matplotlib inline

Medium post on working with gtfs realtime data: https://medium.com/@BennettGarner/parsing-gtfs-format-transit-data-in-real-time-with-python-3a528ba7aab7

And the repo with actual code: https://github.com/bennett39/mta-real-time-data/blob/master/gtfs.py

GTFS reference material: https://developers.google.com/transit/gtfs-realtime/reference/

In [37]:
df_stops = pd.read_csv('data\google_transit\stops.csv')

In [3]:
api_key = '230ffa803afc2f7fa526a3e39982256d'

In [4]:
url_123456S = f'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=1'
url_ACEHS = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=26'
url_NQRW = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=16'
url_BDFM = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=21'
url_L = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=2'
url_G = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=31'
url_JZ = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=36'
url_7 = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=51'

In [5]:
# list of unique URLs for each train line: http://datamine.mta.info/list-of-feeds
url = url_123456S

In [123]:
# make request
response = requests.get(url)
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

173208

In [128]:
type(feed.entity)

google.protobuf.pyext._message.RepeatedCompositeContainer

In [124]:
feed.entity[0]

id: "000001"
trip_update {
  trip {
    trip_id: "110900_1..S"
    start_date: "20190820"
    route_id: "1"
  }
  stop_time_update {
    arrival {
      time: 1566343649
    }
    departure {
      time: 1566343649
    }
    stop_id: "138S"
  }
  stop_time_update {
    arrival {
      time: 1566343739
    }
    departure {
      time: 1566343739
    }
    stop_id: "139S"
  }
  stop_time_update {
    arrival {
      time: 1566343859
    }
    stop_id: "142S"
  }
}

In [130]:
feed.entity[0].trip_update.stop_time_update[0].stop_id

'138S'

In [125]:
one_line = []
two_line = []
three_line = []
four_line = []
five_line = []
six_line = []
s_line = []

for item in feed.entity:
    #Get stop name (need to check that each field exists to avoid errors)
    if not item.HasField('trip_update'):
        continue
    else:
        stop = item.trip_update.stop_time_update[0].stop_id
    
    #Get train line
    if not item.HasField('trip_update'):
        continue
    else:
        line = item.trip_update.trip.route_id
    
    #Append stop name to correct list
    if line == '1':
        one_line.append(stop)
    elif line == '2':
        two_line.append(stop)
    elif line == '3':
        three_line.append(stop)
    elif line == '4':
        four_line.append(stop)
    elif line == '5':
        five_line.append(stop)
    elif line == '6':
        six_line.append(stop)
    elif line == 'S':
        s_line.append(stop)

In [126]:
one_line_update = []

for stop in one_line:
    station_name = df_stops[df_stops['stop_id'] == stop]['stop_name'].to_string(index=False)
    if station_name not in one_line_update:
        one_line_update.append(station_name)

In [127]:
one_line_update

['WTC Cortlandt',
 'Van Cortlandt Park - 242 St',
 '238 St',
 'Chambers St',
 '231 St',
 'Franklin St',
 'Marble Hill - 225 St',
 'Dyckman St',
 'Christopher St - Sheridan Sq',
 '181 St',
 '28 St',
 'Times Sq - 42 St',
 '137 St - City College',
 '59 St - Columbus Circle',
 '96 St',
 '86 St',
 '79 St',
 '66 St - Lincoln Center',
 '116 St - Columbia University',
 '23 St',
 '215 St',
 'South Ferry']